In [92]:
import sqlite3
import pandas as pd
file = 'C:/Users/gdoku/YandexDisk/Компания/07 Учётные регистры и первичные документы/accounting.db'
conn = sqlite3.connect(file)
c = conn.cursor()

tables = c.execute(
    "SELECT `name`, `sql` FROM `sqlite_master` WHERE type='table'").fetchall()

# Создаём временные таблицы
for table in tables:
    c.execute(
        "CREATE TABLE {} AS SELECT * FROM {}".format(table[0]+'_temp', table[0]))

# Удаляем старые таблицы
for table in tables:
    if 'temp' not in table[0]:
        c.execute("DROP TABLE {}".format(table[0]))

# Создаём таблицы
c.execute('CREATE TABLE IF NOT EXISTS sc_types (\
              sc_type_id INTEGER,\
              sc_type TEXT NOT NULL UNIQUE,\
              doc INTEGER NOT NULL CHECK (doc=0 OR doc=1) DEFAULT 0, \
              PRIMARY KEY (sc_type_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS accounts ( \
              account INTEGER, \
              account_name TEXT NOT NULL, \
              sc_type_id_1 INTEGER REFERENCES sc_types (sc_type_id), \
              sc_type_id_2 INTEGER REFERENCES sc_types (sc_type_id), \
              PRIMARY KEY (account)) WITHOUT ROWID'
          )
c.execute('CREATE TABLE IF NOT EXISTS document_types ( \
              document_type_id INTEGER, \
              document_type TEXT NOT NULL UNIQUE, \
              PRIMARY KEY (document_type_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS documents ( \
              document_id INTEGER, \
              document_type_id INTEGER NOT NULL REFERENCES document_types (document_type_id),\
              document_date TEXT CHECK (date(document_date) IS NOT NULL OR NULL), \
              document_number TEXT, \
              comment TEXT, \
              PRIMARY KEY (document_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS subcounts ( \
              subcount_id INTEGER, \
              sc_type_id INTEGER NOT NULL REFERENCES sc_types (sc_type_id), \
              sc_name TEXT, \
              document_id INTEGER REFERENCES documents (document_id),\
              archive INTEGER NOT NULL CHECK (archive=0 OR archive=1) DEFAULT 0, \
              PRIMARY KEY (subcount_id))'
          )
c.execute('CREATE TABLE IF NOT EXISTS operations ( \
            operation_id INTEGER,\
            value_date TEXT CHECK (date(value_date) IS NOT NULL),\
            creation_date TEXT NOT NULL CHECK (datetime(creation_date) IS NOT NULL) DEFAULT \'now\', \
            document_id INTEGER NOT NULL REFERENCES documents (document_id), \
            operation_type_id INTEGER, \
            registered INTEGER NOT NULL CHECK (registered=0 OR registered=1) DEFAULT 0, \
            PRIMARY KEY (operation_id))'
          )
# c.execute('CREATE VIEW IF NOT EXISTS oper_with_doc AS \
#                 SELECT \
#                   operations.operation_id, \
#                   operations.value_date, \
#                   operations.document_id, \
#                   documents.document_id, \
#                   documents.document_type_id, \
#                   documents.document_date, \
#                   documents.document_number, \
#                   documents.comment \
#                 FROM documents INNER JOIN operations ON documents.document_id = operations.document_id'
#           )

# Копируем данные
for table in tables:
    if 'temp' not in table[0]:
        c.execute("INSERT INTO {} SELECT * FROM {}".format(table[0], table[0]+'_temp'))

# Удаляем временные таблицы
for table in tables:
    if 'temp' not in table[0]:
        c.execute("DROP TABLE {}".format(table[0]+'_temp'))

conn.commit()
conn.close()

In [85]:
conn = sqlite3.connect(file)
c = conn.cursor()
# c.execute('ALTER TABLE sc_types \
#             ADD doc INTEGER CHECK (doc is NULL OR doc=1) \
#           ')
data = c.execute(
    "SELECT `name`, `sql`\
  FROM `sqlite_master` \
  WHERE type='table'").fetchall()
#data = pd.DataFrame(data)
for table in data:
    if 'temp' not in table[0]:
        print(table[0])
#    c.execute("DROP TABLE {}".format(table[0]+'_temp'))
conn.close()
data

sc_types
accounts
document_types
documents
subcounts
operations


[('sc_types_temp',
  'CREATE TABLE sc_types_temp(\n  sc_type_id INT,\n  sc_type TEXT,\n  doc INT\n)'),
 ('accounts_temp',
  'CREATE TABLE accounts_temp(\n  account INT,\n  account_name TEXT,\n  sc_type_id_1 INT,\n  sc_type_id_2 INT\n)'),
 ('document_types_temp',
  'CREATE TABLE document_types_temp(\n  document_type_id INT,\n  document_type TEXT\n)'),
 ('documents_temp',
  'CREATE TABLE documents_temp(\n  document_id INT,\n  document_type_id INT,\n  document_date TEXT,\n  document_number TEXT,\n  comment TEXT\n)'),
 ('subcounts_temp',
  'CREATE TABLE subcounts_temp(\n  subcount_id INT,\n  sc_type_id INT,\n  sc_name TEXT,\n  document_id INT,\n  archive INT\n)'),
 ('operations_temp',
  'CREATE TABLE operations_temp(\n  operation_id INT,\n  value_date TEXT,\n  creation_date TEXT,\n  document_id INT,\n  operation_type_id INT,\n  registered INT\n)'),
 ('sc_types',
  'CREATE TABLE sc_types (              sc_type_id INTEGER,              sc_type TEXT NOT NULL UNIQUE,              doc INTEGER N

In [94]:
from tkinter import *

root = Tk()
root.title("Hello World!")
root.geometry('300x40')

def button_clicked():
    print("Hello World!")

def close():
    root.destroy()
    root.quit()

button = Button(root, text="Press Me", command=button_clicked)
button.pack(fill=BOTH)

root.protocol('WM_DELETE_WINDOW', close)

root.mainloop()

Hello World!
Hello World!
Hello World!


In [13]:
# Создадим таблицы данных sc_types
c.execute('CREATE TABLE IF NOT EXIST sc_types \
            (sc_type TEXT NOT NULL UNIQUE)'        
conn.commit()
conn.close()

SyntaxError: invalid syntax (<ipython-input-13-73bc24d9518c>, line 5)

In [5]:
data = cur.execute('SELECT * FROM stocks').fetchall()
# data = data.fetchall()
conn.close()
data

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 ('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 ('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 ('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 ('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 ('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 ('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]

In [4]:
conn = sqlite3.connect('example.db')
cur = conn.cursor()
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")
data = cur.execute('SELECT rowid, * FROM stocks').fetchall()
conn.commit()
conn.close()

In [6]:
data

[(1, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 (2, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 (3, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 (4, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 (5, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14),
 (6, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)]

In [22]:
import pandas as pd

In [23]:
pd.DataFrame(data)

,0,1,2,3,4,5
0,1,2006-01-05,BUY,RHAT,100.0,35.14
1,2,2006-01-05,BUY,RHAT,100.0,35.14
2,3,2006-01-05,BUY,RHAT,100.0,35.14
3,4,2006-01-05,BUY,RHAT,100.0,35.14
4,5,2006-01-05,BUY,RHAT,100.0,35.14


In [32]:
# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

OperationalError: database is locked

In [34]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
data = c.execute('SELECT * FROM stocks')
conn.close()

OperationalError: database is locked

In [ ]:
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

In [12]:
data = c.execute('SELECT * FROM stocks')
data.fetchall()

OperationalError: database is locked

In [33]:
c.close()
conn.close()

In [25]:
conn = sqlite3.connect('example.db')

In [26]:
c = conn.cursor()

In [28]:
data = c.execute('SELECT * FROM stocks')

OperationalError: database is locked

In [27]:
conn.commit()